## Single Consumer - Multiple Publisher
### get()

`get multiple items not more than the specified batch size` **and** `get the items within the timeout`

In [1]:
from queue import Queue, Empty

In [2]:
import time

In [3]:
from threading import Thread

In [4]:
from threading import Event

In [5]:
class BatchedQueue:
    def __init__(self, timeout=1., bs=1):
        self.timeout = timeout
        self.bs = bs
        self._queue: Queue = Queue()
        self._result = []
        self._event = Event()
        
    def get(self):
        entered_at = time.time()
        timeout = self.timeout
        bs = self.bs
        
        if self._queue.qsize()>=bs:
            return [self._queue.get_nowait() for _ in range(bs)]

        while self._event.wait(timeout - (time.time()-entered_at)) and self._queue.qsize() < bs:
            True

        result = []
        try:
            for _ in range(bs):
                result.append(self._queue.get_nowait())
            return result
        except Empty:
            return result
        
    def put(self, item):
        self._queue.put(item)
        if self._event.is_set() and self.size >= self.bs:
            self._event.set()

    @property
    def size(self):
        return self._queue.qsize()

In [7]:
q = BatchedQueue(timeout=2, bs=4)

q.put(1)
q.size

0

In [9]:
t0 = time.time()
print("size", q.size)
q.get()
print("size", q.size)
time.time() - t0

size 1
size 0


2.005971908569336

## Test with a publisher

In [6]:
import random

In [72]:
q = BatchedQueue(timeout=2, bs=4)

def publisher():
    for i in range(16):
        time.sleep(random.randint(0, 1))
        q.put(random.randint(1000, 100000))

thread1 = Thread(target=publisher, daemon=True)
thread2 = Thread(target=publisher, daemon=True)
thread3 = Thread(target=publisher, daemon=True)

thread1.start()
thread2.start()
thread3.start()

In [73]:
q.size

1

In [74]:
for i in range(12):
    t0 = time.time()
    items = q.get()
    print(items)
    t1 = time.time()
    print(f"consumed in {t1-t0:.2f}")

[41763, 37187, 92159, 26374]
consumed in 2.01
[23742, 25296, 38162, 63866]
consumed in 0.00
[59609, 44815, 63455, 36995]
consumed in 0.00
[95961, 8782, 36369, 26241]
consumed in 0.00
[26588, 25025, 62531, 82674]
consumed in 2.01
[81486, 78603, 97243, 12901]
consumed in 0.00
[98119, 30974, 75759, 66431]
consumed in 0.00
[21387, 66863, 44436, 79741]
consumed in 2.00
[71203, 10048, 65738, 49077]
consumed in 0.00
[50332, 45034, 50593, 7350]
consumed in 2.00
[76572, 49153, 62061, 17380]
consumed in 0.00
[68097, 98301, 56020, 56934]
consumed in 2.01
